In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/NLP/Pak Suzuki Car Reviews.csv')

df.head(10)
# convert the date column into date time object
# clean the review column using RegEx library

,Unnamed: 0,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,0,2023 Suzuki Bolan,1,"Nov 30, 2023",\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1
1,1,2022 Suzuki Swift GLX CVT,4,"Nov 29, 2023","\nA good looking car with best fuel ? economy,...",5,4,5,4,5
2,2,2022 Suzuki Cultus Auto Gear Shift,5,"Nov 26, 2023",\nfor me i would must buy because its will sav...,5,5,5,5,5
3,3,2007 Suzuki Liana RXi (CNG),5,"Nov 12, 2023",\nI have used it for more than 5 years. litera...,5,5,5,5,5
4,4,2022 Suzuki Alto VX,4,"Nov 09, 2023",\nExterior: 10/10 on design but the quality us...,5,5,5,4,5
5,5,2020 Suzuki Xbee MZ,5,"Nov 03, 2023",\nIts like a mini SUV great design and feature...,5,5,5,5,5
6,6,2023 Suzuki Alto VXL AGS,3,"Oct 30, 2023",\nnot bad but their is some features missing i...,1,3,5,3,3
7,7,2019 Suzuki Wagon R VXR,4,"Oct 26, 2023",\nPerfect car in under 25 lacPerfect MileageQu...,4,4,5,5,5
8,8,2023 Suzuki Swift GLX CVT,4,"Oct 25, 2023",\nLove the vehicle since its feature packed wi...,5,3,5,5,5
9,9,2021 Suzuki Bolan VX Euro II,1,"Oct 25, 2023",\nis mai do marz boht kharab hain ek heatup or...,2,1,2,1,1


In [22]:
# Removing Unnamed:0 column
df.drop(columns=['Unnamed: 0'], inplace=True)

In [23]:
# Converting Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])


In [24]:
# cleaning the Review column
df['Review'] = df['Review'].str.replace('\n', ' ').str.strip()

In [25]:
# checking for missing values
df.isnull().sum()

Model              0
Overall_rating     0
Date               0
Review             0
Style              0
Comfort            0
Fuel_Economy       0
Performance        0
Value_for_Money    0
dtype: int64

In [26]:
# year column
df['Year'] = df['Date'].dt.year

In [33]:
df

,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year
0,2023 Suzuki Bolan,1,2023-11-30,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1,2023
1,2022 Suzuki Swift GLX CVT,4,2023-11-29,"A good looking car with best fuel ? economy,Dr...",5,4,5,4,5,2023
2,2022 Suzuki Cultus Auto Gear Shift,5,2023-11-26,for me i would must buy because its will save ...,5,5,5,5,5,2023
3,2007 Suzuki Liana RXi (CNG),5,2023-11-12,I have used it for more than 5 years. literall...,5,5,5,5,5,2023
4,2022 Suzuki Alto VX,4,2023-11-09,Exterior: 10/10 on design but the quality used...,5,5,5,4,5,2023
...,...,...,...,...,...,...,...,...,...,...
1094,Suzuki Alto,4,2009-08-21,"I am a salaried person ,I think Alto is a suit...",3,3,4,4,4,2009
1095,Suzuki Cultus,4,2009-06-17,Very good economical car specially for Pakista...,4,3,3,3,4,2009
1096,Suzuki Cultus VXLi (CNG),3,2009-11-13,its a nice car but the price of a car is very ...,3,2,3,2,1,2009
1097,Suzuki Baleno,4,2009-06-05,I have had a Baleno since 2004. Before this I ...,3,3,3,5,5,2009


In [28]:
df.to_csv('Pak Suzuki')

In [29]:
!pip install transformers~=4.31.0
!pip install -q langchain
!pip install -q torch
!pip install -q sentence-transformers
!pip install -q faiss-cpu

In [8]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [30]:
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path='/content/Pak Suzuki'

loader = CSVLoader(file_path)

data = loader.load()

In [73]:
# Define the path to the pre-trained model you want to use
modelPath = "google/flan-t5-base"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

.gitattributes:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/13.4k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [74]:
db = FAISS.from_documents(data, embeddings)

In [75]:
question = "how is alto 2022?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

: 718
Model: Suzuki Liana RXi
Overall_rating: 1
Date: 2015-04-22
Review: obligations, you like to lower the eye rate and perhaps afford a expensive vehicle.So when you find yourself negotiating using your Pasco, WA motor dealer you wish to follow selected guidelines implemented almost universally by individuals purchasing brand new cars.1st, as it is likely you know, no truck price is defined in gemstone.The Pasco, Washington motor dealers may well slap this large amount stickers for the car, but if it implies the big difference between an important sale with out sale, they probably will be ready to drop an economical amount heli-copter flight price.This is not going to decrease the amount of money you'll need to pay from month to month, but overall you'll be saving considerably.Even if you ever won't benefit for a while, the long-term benefit may be valued at a very little haggling now considering the Pasco, WA auto company.It will furnish you a number of extra extra income down the r

In [76]:
# Create a tokenizer object by loading the pretrained "Intel/dynamic_tinybert" tokenizer.
tokenizer = AutoTokenizer.from_pretrained("Intel/dynamic_tinybert")

# Create a question-answering model object by loading the pretrained " Intel/dynamic_tinybert" model.
model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

In [77]:
# Specify the model name you want to use
model_name = "google/flan-t5-base"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answer,
    model_kwargs={"temperature": 0, "max_length": 256},
)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [78]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or documents from the database.
retriever = db.as_retriever()

In [79]:
docs = retriever.get_relevant_documents("What are the reviews regardinng Suzuki Swift GLX CVT?")
print(docs[0].page_content)

: 869
Model: Suzuki Cultus EURO II
Overall_rating: 1
Date: 2012-10-13
Review: My review abt CULTUS Euro II is negative coz its not of the standard on which suzuki manufactured the last models of CULTUS.Its comfort is not up to the mark,I would suggest suzuki to ponder upon their newly manufactured CULTUS.This is all i wana say
Style: 1
Comfort: 1
Fuel_Economy: 2
Performance: 2
Value_for_Money: 3
Year: 2012


In [80]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [81]:
question = "What are the reviews regarding Suzuki Alto in year 2018?"
result = qa.run({"query": "What are the reviews regarding Suzuki Alto in year 2018?"})
print(result["result"])

ValueError: Context information is below. 
------------
: 766
Model: 2014 Suzuki Swift DLX 1.3
Overall_rating: 2
Date: 2014-06-27
Review: People who jump from Mehran, Alto or Cultus, love this car. But this is not as good as it is being appreciated.No doubt, exterior shape is awsome, but inside, rear seats are space less actually. Suzuki actually placed LIANA's engine in it, 1328 cc. That falls in the category of 1300 to 1500 cc, so registration of a car without trunk, is as much more than Corrolla XLI or Honda City.So do u think a car made by Suzuki having engine 1328 cc will consume less fuel than 1300 cc cars?Bad idea....
Style: 4
Comfort: 3
Fuel_Economy: 2
Performance: 3
Value_for_Money: 2
Year: 2014
------------
Given the context information and not prior knowledge, answer the question: What are the reviews regarding Suzuki Alto in year 2018?
 argument needs to be of type (SquadExample, dict)

In [48]:
question = "What are the reviews regarding Suzuki Alto in year 2018?"
result = qa.run({"query": question})
print(result["result"])


ValueError: Context information is below. 
------------
: 882
Model: Suzuki Mehran VX Euro II
Overall_rating: 0
Date: 2012-09-07
Review: Its a good car in the sense that it is most selling. Its repair work is relatively cheap. Its mass production has thrown many good cars out of market . Cars such as Chevy JOy , Cherry QQ have not been successful in the Pak market despite good build quality and price.
Style: 0
Comfort: 0
Fuel_Economy: 0
Performance: 0
Value_for_Money: 0
Year: 2012
------------
Given the context information and not prior knowledge, answer the question: What are the reviews regarding Suzuki Alto in year 2018?
 argument needs to be of type (SquadExample, dict)